# Differentiable Shallow Water PDE Solver - Benchmarks


[![GitHubBadge]][GitHubLink] [![ColabBadge]][ColabLink]


[ColabBadge]: https://colab.research.google.com/assets/colab-badge.svg "Run notebook in Google Colab"
[ColabLink]: https://colab.research.google.com/gist/vojtamolda/bd85033cf62877e6f8ada68b8bbb32a0/Benchmarks.ipynb

[GitHubBadge]: https://img.shields.io/badge/|-Edit_on_GitHub-green.svg?logo=github "Edit notebook's source code on GitHub"
[GitHubLink]: https://gist.github.com/vojtamolda/bd85033cf62877e6f8ada68b8bbb32a0#file-benchmarks-ipynb

In [1]:
// Clone Gist
%system rm --recursive --force Solver
%system git clone https://gist.github.com/bd85033cf62877e6f8ada68b8bbb32a0.git Solver

// Install Packages
%install-swiftpm-flags -c release
%install-location /swift/packages

%install '.package(url: "https://github.com/google/swift-benchmark", .branch("master"))' Benchmark
%install '.package(url: "https://github.com/vojtamolda/Plotly.swift.git", from: "0.3.1")' Plotly
%install '.package(url: "https://github.com/t-ae/Swim.git", from: "3.9.0")' Swim

// Clear Output
print("\u{001B}[2J")

In [2]:
import TensorFlow
import Benchmark

## Differentiable PDE Solver

The next cell imports three different implementations differentiable shallow water PDE solver from a cloned git repository. The full source code and a readme file can be found in [this GitHub gist](https://gist.github.com/bd85033cf62877e6f8ada68b8bbb32a0.git).


In [3]:
%include "Solver/Solution.swift"
%include "Solver/ArrayLoopSolution.swift"
%include "Solver/TensorLoopSolution.swift"
%include "Solver/TensorSliceSolution.swift"
%include "Solver/TensorConvSolution.swift"

## Benchmarks

The following code runs a simulation of a water surface behavior in a rectangular bathtub. There's an initial "splash" at the begining. The splash generates surface gravity waves that propagate away from the center and reflect off the domain walls. There's three different versions, one for each implementation of the solver.

Implementations that use the `Tensor` type for numerical values also acept the `device` argument. This allows them to run with XLA acceleration.

In [4]:
let n = 256
let duration = 512

#### A - `ArrayLoopSolution`

In [5]:
func splashArrayLoop() {
    var initialWaterLevel = [[Float]](repeating: [Float](repeating: 0.0, count: n), count: n)
    initialWaterLevel[n / 2][n / 2] = 100

    let initialSolution = ArrayLoopSolution(waterLevel: initialWaterLevel)
    _ = [ArrayLoopSolution](evolve: initialSolution, for: duration)
}

#### B - `TensorLoopSolution`

In [6]:
func splashTensorLoop(on device: Device) {
    var initialWaterLevel = Tensor<Float>(zeros: [n, n], on: device)
    initialWaterLevel[n / 2][n / 2] = Tensor<Float>(100, on: device)

    let initialSolution = TensorLoopSolution(waterLevel: initialWaterLevel)
    _ = [TensorLoopSolution](evolve: initialSolution, for: duration)
}

#### C - `TensorSliceSolution`

In [7]:
func splashTensorSlice(on device: Device) {
    var initialWaterLevel = Tensor<Float>(zeros: [n, n], on: device)
    initialWaterLevel[n / 2][n / 2] = Tensor<Float>(100, on: device)

    let initialSolution = TensorSliceSolution(waterLevel: initialWaterLevel)
    _ = [TensorSliceSolution](evolve: initialSolution, for: duration)
}

#### D - `TensorConvSolution`

In [8]:
func splashTensorConv(on device: Device) {
    var initialWaterLevel = Tensor<Float>(zeros: [n, n], on: device)
    initialWaterLevel[n / 2][n / 2] = Tensor<Float>(100, on: device)

    let initialSolution = TensorConvSolution(waterLevel: initialWaterLevel)
    _ = [TensorConvSolution](evolve: initialSolution, for: duration)
}

## Results

Not yet conclusive...

In [9]:
let splashBenchmarks = BenchmarkSuite(name: "Shallow Water PDE Solver",
                                      settings: Iterations(10), WarmupIterations(2)) { suite in
    suite.benchmark("Array Loop") { splashArrayLoop() }

    // This is at least 1000x slower. One can easily grow old while running the benchmark :(
    //suite.benchmark("Tensor Loop") { splashTensorLoop(on: Device.default) }
    //suite.benchmark("Tensor Loop (XLA)") { splashTensorLoop(on: Device.defaultXLA) }

    suite.benchmark("Tensor Slice") { splashTensorSlice(on: Device.default) }
    suite.benchmark("Tensor Slice (XLA)") { splashTensorSlice(on: Device.defaultXLA) }

    suite.benchmark("Tensor Conv") { splashTensorConv(on: Device.default) }
    suite.benchmark("Tensor Conv (XLA)") { splashTensorConv(on: Device.defaultXLA) }
}

In [10]:
Benchmark.main([splashBenchmarks])

running Shallow Water PDE Solver: Array Loop... done! (363294.84 ms)
running Shallow Water PDE Solver: Tensor Slice... done! (9566.95 ms)
running Shallow Water PDE Solver: Tensor Slice (XLA)... done! (17147.61 ms)
running Shallow Water PDE Solver: Tensor Conv... done! (83571.66 ms)
running Shallow Water PDE Solver: Tensor Conv (XLA)... done! (19268.14 ms)

name                                        time               std        iterations warmup        
---------------------------------------------------------------------------------------------------
Shallow Water PDE Solver.Array Loop         30268327673.500 ns ±   0.40 %         10 60451444387 ns
Shallow Water PDE Solver.Tensor Slice         741456533.500 ns ±   4.13 %         10  2012324321 ns
Shallow Water PDE Solver.Tensor Slice (XLA)  1359265173.500 ns ±   2.19 %         10  3534559405 ns
Shallow Water PDE Solver.Tensor Conv         7026560678.500 ns ±   2.92 %         10 13381071502 ns
Shallow Water PDE Solver.Tensor Conv (XLA